In [1]:
import pandas as pd
import requests
import json
from urllib import parse
import matplotlib as plt
import numpy as np
import os
import time
import warnings


warnings.filterwarnings(action='ignore') 

In [2]:
apiKey = 'RGAPI-074edd7a-f351-4b1c-ba5e-c61af955eb6b'
queue = "/RANKED_SOLO_5x5"

In [3]:
def getID(tier, rank):
    page = 1
    sleepCount = 0
    df = pd.DataFrame()

    while True:
        url = 'https://kr.api.riotgames.com/lol/league/v4/entries' + queue + '/' + tier + '/' + rank + '?page=' +str(page)+ '&api_key=' + apiKey 
        names = requests.get(url)
        statusCode = names.status_code
        if statusCode == 200:
            pass

        elif statusCode == 429:
            while True: 
                if statusCode == 429:
                    sleepCount += 1
                    print(f'120초 쉬기 / sleepCount: {sleepCount} ID 수: {len(df)}, page: {page}')
                    time.sleep(120)
                    break
                elif statusCode == 200:
                    break

        names = names.json()

        try:
            names = pd.DataFrame(names)[['tier', 'rank', 'summonerId', 'summonerName']]
            df = pd.concat([df, names])

        except:
            break

        page += 1

    df.to_csv('./' + tier + rank + 'id.csv', index=False, encoding= 'UTF-8-sig')

In [35]:
getID('DIAMOND', 'IV')

120초 쉬기 / sleepCount: 1 ID 수: 20500, page: 101


In [4]:
diamond1 = pd.read_csv('./DIAMONDIid.csv')
diamond2 = pd.read_csv('./DIAMONDIIid.csv')
diamond3 = pd.read_csv('./DIAMONDIIIid.csv')
diamond4 = pd.read_csv('./DIAMONDIVid.csv')

In [5]:
diamond3.summonerName

0               가락사냥꾼김주원
1               못한다고하면울어
2        llIIIllIlllIlIl
3                박보영 귀욤이
4             jikbakgurl
              ...       
17158           닐라은면역입니다
17159            KGGGKOR
17160           정글 잘하구싶다
17161              싸지일러스
17162       My style 조이현
Name: summonerName, Length: 17163, dtype: object

In [6]:
def getMatches(username):
    id = parse.quote(username) # 아이디를 URL 인코딩
    url = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/' + id +'?api_key=' + apiKey #puuid값을 가져오기 위한 주소
    names = requests.get(url)

    statusCode = names.status_code

    if statusCode == 200:
        pass

    elif statusCode == 429:
        while True: 
            if statusCode == 429:
                print(f'120초 쉬기 / puuid 수집')
                time.sleep(120)
                break
            elif statusCode == 200:
                break


    names = names.json()
    # print(names)
    puuid = names['puuid'] # 해당 유저의 puuid 값 가져오기

    # 랭크 n경기 매치아이디 가져오기
    n = str(10)
    rankUrl = 'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/' + puuid + '/ids?queue=420&type=ranked&start=0&count='+n+'&api_key='+ apiKey
    r = requests.get(rankUrl)
    if statusCode == 200:
        pass

    elif statusCode == 429:
        while True: 
            if statusCode == 429:
                print(f'120초 쉬기 / 매치데이터 수집')
                time.sleep(120)
                break
            elif statusCode == 200:
                break

    r = r.json()

    rankId = r
    return rankId

In [387]:
getMatches(diamond1.summonerName[38]) 

['KR_6490245708',
 'KR_6478903991',
 'KR_6478893094',
 'KR_6468066647',
 'KR_6466865045',
 'KR_6457102238',
 'KR_6456811326',
 'KR_6456347995',
 'KR_6455543053',
 'KR_6455529017']

In [7]:
diamond3.loc[0:199]

,tier,rank,summonerId,summonerName
0,DIAMOND,III,1nICuLJ_ge3P5WSC0lmr03EyUdWFHt-pYSJOFiE41EUOMJzE,가락사냥꾼김주원
1,DIAMOND,III,8_VJv8UWTojRfRU2iPFRGCFGC-cmzvxGZ-wQN7L_DhC8HH88,못한다고하면울어
2,DIAMOND,III,pbUEb33jj22RvuLDl4ryBkKKRJc4nGi183v8Ca86VpBb09CF,llIIIllIlllIlIl
3,DIAMOND,III,Ukypmw1rVcSQliHHfd7et1krGhVf0_T-xr_WTSOm3jNYztCg,박보영 귀욤이
4,DIAMOND,III,IzcwVS-qCEGy8P67eJRKOVLSVoMeTnpvOMrcHtmwJYXqGBT2,jikbakgurl
...,...,...,...,...
195,DIAMOND,III,TBkKEFihwExzYu8Muj3K_p9jMyUFv3dVbABl6AWrD0Aqxgfq,그저 신지드
196,DIAMOND,III,W7Qz7CExGUzCiabXPIyTEMwhjO5UkvpP0q3WVVMGrbPxeH4,표범 너무 귀여워
197,DIAMOND,III,dp2N1UuIzuVnI-b4nZGx5qheGgQ_V-tpx9RyuCrIuo_tngs,HaeDeun
198,DIAMOND,III,cg4Wmqag94BPfCMrhHBBk0Pk5WlP6huhJnlTZFu2ETNSQZtk,kacsgyo


In [9]:
data = diamond3.loc[1000:1199]

result = [0] * len(data)
for i in range(len(data)):
    try: 
        result[i] = getMatches(data.summonerName[i])
    except:
        pass

diamond3_result = result

In [352]:
match_list = len(diamond3_result) * [0] * 10

for i in range(len(diamond3_result)):
    for j in range(10):
        try:
            match_list[(i*10) + j] = diamond3_result[i][j]
        except:
            pass

matches = pd.Series(match_list).drop_duplicates().dropna().reset_index(drop=True)
matches.to_csv('./matches_diamond3_200_800.csv', index=False)

In [353]:
matches = pd.read_csv('./matches_diamond3_200_800.csv')

In [354]:
matchresult = [0] * len(matches)
timeline = [0] * len(matches)
matchdata = pd.DataFrame([matchresult, timeline]).T
matchdata = pd.concat([matches, matchdata], axis=1)
matchdata.columns = ['matches', 'matchresult', 'timeline']

In [355]:
matchdata.matches[0]

'0'

In [359]:
def getMatchData(dataFrame):
    version = '13.9.506.4846'

    for i in range(len(dataFrame)):
        matchid = dataFrame.matches[i]
        url = 'https://asia.api.riotgames.com/lol/match/v5/matches/' + matchid +'?api_key=' + apiKey 
        matchresult = requests.get(url)

        statusCode = matchresult.status_code
        if statusCode == 200:
            pass

        elif statusCode == 429:
            while True: 
                if statusCode == 429:
                    print(f'120초 쉬기 / 현재 {i}번째')
                    time.sleep(120)
                    break
                elif statusCode == 200:
                    break\
                    
        matchresult = matchresult.json()

        try:
            if matchresult['info']['gameVersion'] != version:   # 13.9 패치버전이 아닐 경우 무시
                continue
        
            pd.json_normalize(matchresult['info']['participants']).to_csv(f'./matchdata2/diamond3_{i}.csv', encoding= 'UTF-8-sig')
            pd.json_normalize(matchresult['info']['teams']).to_csv(f'./matchdata_team2/diamond3_{i}.csv', encoding= 'UTF-8-sig')
        
        except: continue

        url = 'https://asia.api.riotgames.com/lol/match/v5/matches/' + matchid + '/timeline?api_key=' + apiKey 
        timeline = requests.get(url)

        statusCode = timeline.status_code
        if statusCode == 200:
            pass

        elif statusCode == 429:
            while True: 
                if statusCode == 429:
                    print(f'120초 쉬기 / 현재 {i}번째')
                    time.sleep(120)
                    break
                elif statusCode == 200:
                    break

        timeline = timeline.json()

        try:
            pd.json_normalize(timeline['info']['frames']).to_csv(f'./timeline2/diamond3_{i}.csv', encoding= 'UTF-8-sig')
        except:
            continue

In [360]:
getMatchData(matchdata)

120초 쉬기 / 현재 56번째
120초 쉬기 / 현재 119번째
120초 쉬기 / 현재 174번째
120초 쉬기 / 현재 234번째
120초 쉬기 / 현재 298번째
120초 쉬기 / 현재 358번째
120초 쉬기 / 현재 445번째
120초 쉬기 / 현재 514번째
120초 쉬기 / 현재 587번째
120초 쉬기 / 현재 646번째
120초 쉬기 / 현재 709번째
120초 쉬기 / 현재 771번째
120초 쉬기 / 현재 842번째
120초 쉬기 / 현재 912번째
120초 쉬기 / 현재 989번째
120초 쉬기 / 현재 1056번째
120초 쉬기 / 현재 1123번째
120초 쉬기 / 현재 1186번째
120초 쉬기 / 현재 1238번째
120초 쉬기 / 현재 1309번째
120초 쉬기 / 현재 1372번째
120초 쉬기 / 현재 1432번째
120초 쉬기 / 현재 1501번째
120초 쉬기 / 현재 1552번째
120초 쉬기 / 현재 1617번째
120초 쉬기 / 현재 1671번째
120초 쉬기 / 현재 1743번째
120초 쉬기 / 현재 1803번째
120초 쉬기 / 현재 1862번째
120초 쉬기 / 현재 1916번째
120초 쉬기 / 현재 1975번째
120초 쉬기 / 현재 2034번째
120초 쉬기 / 현재 2097번째
120초 쉬기 / 현재 2162번째
120초 쉬기 / 현재 2224번째
120초 쉬기 / 현재 2282번째
120초 쉬기 / 현재 2333번째
120초 쉬기 / 현재 2400번째
120초 쉬기 / 현재 2477번째
120초 쉬기 / 현재 2549번째
120초 쉬기 / 현재 2617번째
120초 쉬기 / 현재 2686번째
120초 쉬기 / 현재 2742번째
120초 쉬기 / 현재 2803번째
120초 쉬기 / 현재 2865번째
120초 쉬기 / 현재 2926번째
120초 쉬기 / 현재 2994번째
120초 쉬기 / 현재 3074번째
120초 쉬기 / 현재 3143번째
120초 쉬기 / 현재 3213번째
120초 쉬기 / 현재 327

In [73]:
pd.DataFrame(matchresult['info']['participants'][0])

,allInPings,assistMePings,assists,baitPings,baronKills,basicPings,bountyLevel,challenges,champExperience,champLevel,...,turretKills,turretTakedowns,turretsLost,unrealKills,visionClearedPings,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win
12AssistStreakCount,0,8,9,0,0,0,0,0.0,24459,18,...,2,2,10,0,0,46,4,5,22,False
abilityUses,0,8,9,0,0,0,0,597.0,24459,18,...,2,2,10,0,0,46,4,5,22,False
acesBefore15Minutes,0,8,9,0,0,0,0,0.0,24459,18,...,2,2,10,0,0,46,4,5,22,False
alliedJungleMonsterKills,0,8,9,0,0,0,0,0.0,24459,18,...,2,2,10,0,0,46,4,5,22,False
baronTakedowns,0,8,9,0,0,0,0,0.0,24459,18,...,2,2,10,0,0,46,4,5,22,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wardTakedowns,0,8,9,0,0,0,0,5.0,24459,18,...,2,2,10,0,0,46,4,5,22,False
wardTakedownsBefore20M,0,8,9,0,0,0,0,2.0,24459,18,...,2,2,10,0,0,46,4,5,22,False
wardsGuarded,0,8,9,0,0,0,0,0.0,24459,18,...,2,2,10,0,0,46,4,5,22,False
statPerks,0,8,9,0,0,0,0,NaN,24459,18,...,2,2,10,0,0,46,4,5,22,False


In [125]:
pd.json_normalize(matchresult['info']['participants'])

,allInPings,assistMePings,assists,baitPings,baronKills,basicPings,bountyLevel,champExperience,champLevel,championId,...,challenges.junglerKillsEarlyJungle,challenges.killsOnLanersEarlyJungleAsJungler,challenges.fastestLegendary,challenges.earliestBaron,challenges.highestChampionDamage,challenges.shortestTimeToAceFromFirstTakedown,challenges.soloTurretsLategame,challenges.teleportTakedowns,challenges.highestWardKills,challenges.fasterSupportQuestCompletion
0,0,8,9,0,0,0,0,24459,18,897,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,8,1,0,0,0,21446,18,238,...,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,3,7,0,0,0,0,24538,18,84,...,NaN,NaN,1740.022868,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,1,8,0,0,0,0,20109,18,498,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,17,15,2,0,0,0,14913,16,412,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0,11,5,0,0,0,0,26868,18,24,...,NaN,NaN,NaN,1413.038537,1.0,48.54512,1.0,2.0,NaN,NaN
6,0,0,17,0,3,0,2,26239,18,76,...,0.0,1.0,NaN,1413.038537,NaN,NaN,NaN,NaN,1.0,NaN
7,0,7,13,0,0,0,1,22816,18,711,...,NaN,NaN,NaN,1413.038537,NaN,48.54512,NaN,NaN,NaN,NaN
8,1,0,13,0,0,0,4,22269,18,202,...,NaN,NaN,NaN,1413.038537,NaN,NaN,NaN,NaN,NaN,NaN
9,0,0,17,0,0,0,0,18428,18,74,...,NaN,NaN,NaN,1413.038537,NaN,NaN,NaN,NaN,NaN,1.0


In [364]:
len(os.listdir('./matchdata2/'))

2217

In [365]:
len(os.listdir('./timeline2/'))

2198

In [366]:
matchresults = os.listdir('./matchdata2/')
timelines = os.listdir('./timeline2/')

namesList = pd.Series(timelines + matchresults)  
for i in namesList.drop_duplicates()[len(timelines):].to_list(): # 중복되지 않는 데이터 찾기
    matchresults.remove(i)
    
namesList = pd.Series(matchresults + timelines)  
for i in namesList.drop_duplicates()[len(matchresults):].to_list(): # 중복되지 않는 데이터 찾기
    timelines.remove(i)

In [370]:
matchresult = pd.DataFrame()
timeline = pd.DataFrame()

for i in range(len(matchresults)):
    matchresult = pd.concat([matchresult, pd.read_csv('./matchdata2/' + matchresults[i])])
    timeline = pd.concat([timeline, pd.read_csv('./timeline2/' + timelines[i])])

In [371]:
matchresult.to_csv('./matchresult2.csv', encoding='UTF-8-sig', index=False)
timeline.to_csv('./timeline2.csv', encoding='UTF-8-sig', index=False)

In [347]:
matchresult.championName.value_counts()

Ezreal      393
Jinx        278
Aphelios    272
Thresh      257
LeeSin      239
           ... 
Quinn         9
Sona          6
Corki         5
Skarner       5
Ivern         4
Name: championName, Length: 163, dtype: int64

In [349]:
matchresult.win.value_counts()

False    5245
True     5245
Name: win, dtype: int64

In [344]:
matchresult.teamPosition

0        TOP
1     JUNGLE
2     MIDDLE
3     BOTTOM
4    UTILITY
      ...   
5        TOP
6     JUNGLE
7     MIDDLE
8     BOTTOM
9    UTILITY
Name: teamPosition, Length: 10490, dtype: object

In [257]:
timeline.events.iloc[3]

"[{'levelUpType': 'NORMAL', 'participantId': 6, 'skillSlot': 3, 'timestamp': 120319, 'type': 'SKILL_LEVEL_UP'}, {'level': 2, 'participantId': 8, 'timestamp': 122083, 'type': 'LEVEL_UP'}, {'levelUpType': 'NORMAL', 'participantId': 8, 'skillSlot': 1, 'timestamp': 122781, 'type': 'SKILL_LEVEL_UP'}, {'itemId': 2003, 'participantId': 5, 'timestamp': 124050, 'type': 'ITEM_DESTROYED'}, {'level': 2, 'participantId': 6, 'timestamp': 131421, 'type': 'LEVEL_UP'}, {'level': 3, 'participantId': 2, 'timestamp': 131824, 'type': 'LEVEL_UP'}, {'levelUpType': 'NORMAL', 'participantId': 2, 'skillSlot': 2, 'timestamp': 134125, 'type': 'SKILL_LEVEL_UP'}, {'levelUpType': 'NORMAL', 'participantId': 6, 'skillSlot': 1, 'timestamp': 134125, 'type': 'SKILL_LEVEL_UP'}, {'level': 2, 'participantId': 3, 'timestamp': 135621, 'type': 'LEVEL_UP'}, {'level': 2, 'participantId': 9, 'timestamp': 135754, 'type': 'LEVEL_UP'}, {'level': 2, 'participantId': 10, 'timestamp': 135754, 'type': 'LEVEL_UP'}, {'levelUpType': 'NORMA

In [305]:
matchresult

,Unnamed: 0,allInPings,assistMePings,assists,baitPings,baronKills,basicPings,bountyLevel,champExperience,champLevel,...,challenges.highestChampionDamage,challenges.shortestTimeToAceFromFirstTakedown,challenges.soloTurretsLategame,challenges.teleportTakedowns,challenges.highestWardKills,challenges.fasterSupportQuestCompletion,challenges.baronBuffGoldAdvantageOverThreshold,challenges.thirdInhibitorDestroyedTime,challenges.earliestElderDragon,challenges.hadAfkTeammate
0,0,0,8,9,0,0,0,0,24459,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0,0,8,1,0,0,0,21446,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1,3,7,0,0,0,0,24538,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0,1,8,0,0,0,0,20109,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1,17,15,2,0,0,0,14913,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10485,5,0,0,1,0,0,0,0,7498,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10486,6,0,1,1,0,0,0,0,5579,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10487,7,0,0,0,0,0,0,0,6615,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10488,8,0,0,1,1,0,0,0,5618,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [302]:
# timestamp 단위는 ms 1000 = 1초 / 600000= 10분

s = timeline.events.iloc[2]
json_acceptable_string = s.replace("'", "\"")
# print(json_acceptable_string)
# json_acceptable_string = json.dumps(json_acceptable_string, indent=4)
# print(json_acceptable_string)
# json.loads(s)
json.loads(json_acceptable_string)


[{'creatorId': 2,
  'timestamp': 64097,
  'type': 'WARD_PLACED',
  'wardType': 'YELLOW_TRINKET'},
 {'itemId': 3340,
  'participantId': 2,
  'timestamp': 65459,
  'type': 'ITEM_DESTROYED'},
 {'itemId': 3364,
  'participantId': 2,
  'timestamp': 65459,
  'type': 'ITEM_PURCHASED'},
 {'levelUpType': 'NORMAL',
  'participantId': 8,
  'skillSlot': 3,
  'timestamp': 77549,
  'type': 'SKILL_LEVEL_UP'},
 {'levelUpType': 'NORMAL',
  'participantId': 9,
  'skillSlot': 1,
  'timestamp': 81683,
  'type': 'SKILL_LEVEL_UP'},
 {'levelUpType': 'NORMAL',
  'participantId': 2,
  'skillSlot': 1,
  'timestamp': 89047,
  'type': 'SKILL_LEVEL_UP'},
 {'level': 2, 'participantId': 2, 'timestamp': 100250, 'type': 'LEVEL_UP'},
 {'levelUpType': 'NORMAL',
  'participantId': 3,
  'skillSlot': 1,
  'timestamp': 100283,
  'type': 'SKILL_LEVEL_UP'},
 {'levelUpType': 'NORMAL',
  'participantId': 2,
  'skillSlot': 3,
  'timestamp': 100848,
  'type': 'SKILL_LEVEL_UP'},
 {'levelUpType': 'NORMAL',
  'participantId': 5,
  '

In [14]:
timeline

{'metadata': {'dataVersion': '2',
  'matchId': 'KR_6483930613',
  'participants': ['8flCmJh5Kiv1zKSbkGjiOgDmnSxDvszSQ-ihf7upKwM8C1h-Jl13P02mQWBPjC_zUt3zgwS29RypoA',
   'OqdrlMlDQgzmZM1pQXa--35pI2a0iNSVz_mI3bCZ6US8hKfvHnfEbb400q0tuASdsNhXVlUXfSJYZQ',
   'GM19GkT86dkBIc6tL2z0O1O_-sWuC_w8C5t6a-2YGgLjFrlLcxzt_HTlc2NEYYIXtLnoGf1JHS0vcQ',
   'jxajO2JUsPLrr0GXsw--lYwlRS4OE0FkB0DEUqoaSdOnTnVYP7q-MrFwOBo8giybjBi_JqVYyy6udw',
   'J1YS_DV60yfGRl2zXXMx52_JnDc77HpFo0biFFuseEX-uMo3yvyn_Hx9Mt7sHPsGjZTxi7E2XJvJEA',
   'OqYvwLvH_KC3g5kyoFarLGYzBNjfJDm8RJbGw70krfmtcOzBro4mDBaGDeARV559sgMDYO0I81bahA',
   'WjIqVRXrIQR6Lc_fQ2FzvsO4P70TMcYQk_ZWHdZpkz4nMzVve7isxaijreZEYcqEQKwoeYL1kFrQHQ',
   'x-4u13mnrNb9XzY2V23carHI4YU36JFJNwkHZKb44y_lAK-xcv_8NQAB9EEUZl7pVrAcAdqAqv4tUQ',
   '6PdPExcE9JgBTuz0Rhv72MJd5THQwXR_lXrZLI2u8K3E0cI2QOY4xCl2m9Y508d6AbMNtnu6GG9SLw',
   'igj9GraQHQI68pztSLWLay-lu2q7uMQ24Plkvc-_lt21LScHnf9rNkg9uZUa2HzEH66PQFKVwRpjFA']},
 'info': {'frameInterval': 60000,
  'frames': [{'events': [{'realTim